In [2]:
import wrds
import pandas as pd
import matplotlib.pyplot as plt
import csv
import glob
import os
import numpy as np
import matplotlib.dates as mdates
from numpy.linalg import inv

We want to find a such that the annual volatility of $R_{FUND} = R_{T_Bill}(1-a) + aR_{DIV}$ is 15%. Hence, since T-bills are considered as risk-free and the DIV annual standard deviation is known from 3c: 

$\sigma(R_{FUND}) = a\sigma(R_{DIV}) = 15\% \implies a = \frac{0,15}{\sigma(R_{DIV})}.$  

Note that we will not short the DIV portfolio as the average return is positive, so the only negative option of a is not taken into account.

In [19]:
# Q8a
div_return = pd.read_csv("data/q3/div_return.csv")
div_std = np.sqrt(12)*div_return["DIV_return"].std()
a = 0.15/div_std
print (div_std)

0.13988019275502003
